# Import Libraries

In [1]:
import os
import sys
from importlib import reload

import numpy as np

import rur
from rur import uri, drawer, utool, uhmi, sci

# Test

## load sim raw data

In [44]:
#!!!!!!!!!!!!!!!!!!!!!!!!

nout = 1026
idgal = 8

repo = '/storage6/NewHorizon'
path_in_repo = 'snapshots'
mode = 'nh'

boxrad = 30

#!!!!!!!!!!!!!!!!!!!!!!!!

In [45]:
snap = uri.RamsesSnapshot(repo=repo, 
                          path_in_repo=path_in_repo, 
                          iout=nout, 
                          mode=mode, 
                          longint=False)

#boxtokpc = 1/snap.unit['kpc']#/snap.params['aexp']

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)


In [46]:
mlow = 10**9
mupp = 1e13

gals, pid = uhmi.HaloMaker.load(snap, 
                                galaxy=True, 
                                full_path='/storage6/NewHorizon/galaxy_old',
                                load_parts=True,
                                double_precision=True)
gals.sort(order='m')
gals = gals[(mlow<=gals['mvir'])*(gals['mvir']<mupp)]
print(gals['id'])

[ 334  163  254  198  190  118 3172  100  633  539 3127  289  495  176
 2183  454  138 2196  287 1267  117 1090  999  193  223   55   12  532
  148  531   57   73 2498   45  749  106  265  145  724  971  839  383
  191  580  188  543  300   85  322  135  213  222  723  333  468 1271
  352  351  194 1179   48  547  698  369  159  229  307  123   76  127
   71  372 2667   44   28   56  124  162  168  402  266   87 1077 4642
  211  296  297  450 1055  115  178   80  241  550  886  474  153  818
  257  151   51  119  267   61   90   94  155  103  255   59   72  146
  196  139  120 2951   79  277   69  690  207   83   70  142 4529  202
  200  232  174 1263  212  179 4083  371   86   97   67   50  111    5
   38   15  181   18   88 2557  195   22  116  629  822  137 1141  208
   40   75  104   65   41   84   17 1723  134  166   63   47   52  214
   37 1373  108  107   35   54   29   60 2182   91   46   24  129  990
   66   92  688    3   21 1730  169  149   26   11   68   36   64   13
   23 

In [47]:
gal = gals[gals['id']==idgal]

In [48]:
snap.clear()
snap = uri.RamsesSnapshot(repo=repo, 
                          path_in_repo=path_in_repo, 
                          iout=nout, 
                          mode=mode)

boxtokpc = 1/snap.unit['kpc']#/snap.params['aexp']
snap.set_box_halo(gal, 
                  use_halo_radius=False, 
                  radius=boxrad/boxtokpc)

star = snap.get_part(pname='star')

snap.clear()
snap = uri.RamsesSnapshot(repo=repo, 
                          path_in_repo=path_in_repo, 
                          iout=nout, 
                          mode=mode)
snap.set_box_halo(gal, 
                  use_halo_radius=False, 
                  radius=boxrad/boxtokpc)
snap.get_cell()
cell = snap.cell    

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)
Reading 128 part files (778.5 MiB) in /storage6/NewHorizon/snapshots/output_01026... 
Done (2.252s).
Masking particles... 9008886 / 9450156 (0.9533)
Done (0.715s).
[clear] Clearing memory
	Unlink `psm_10ed4f4e`
[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)
Reading 128 AMR & hydro files (1.304 GiB) in /storage6/NewHorizon/snapshots/output_01026... 
Done (4.461s).
Masking cells... 5582179 / 6660944 (0.8380)
Done (0.462s).


## load param file

In [49]:
repo_par = '/home/jangjk/SKIRT_DIR/rur_dir/'
fname_par = 'parameter_test'

try:
    par = reload(sys.modules[fname_par])
except:
    pass

sys.path.append(repo_par)
par = __import__(fname_par)

In [50]:
np.log10(par.N_phot)

7.301029995663981

## execute SKIRT 

In [51]:
rur.rskirt.utils.execute_SKIRT.make_INSKI(
                                           snap=snap,
                                           star=star,
                                           cell=cell,
                                           param=par,
                                           pos_ctr=[gal['x'],gal['y'],gal['z']],
                                           vel_ctr=[gal['vx'],gal['vy'],gal['vz']],
                                            )

done -- basic setting
done -- galaxy rotation
done -- writing old population
done -- writing young population
done -- writing gas cells
21/07/2023 15:56:30.160   Welcome to SKIRT v9.0 (git a313a94 built on 16/06/2022 at 16:37:32)
21/07/2023 15:56:30.160   Running on gc7 for jangjk
21/07/2023 15:56:30.160   Constructing a simulation from ski file '../nh_ext.ski'...
21/07/2023 15:56:30.487   Starting simulation nh_ext using 40 threads and a single process...
21/07/2023 15:56:30.487   Starting setup...
21/07/2023 15:56:30.487     Panchromatic wavelength regime
21/07/2023 15:56:30.487     With transfer medium
21/07/2023 15:56:30.487     Model and grid symmetry: 3D
21/07/2023 15:56:30.487   BruzualCharlotSEDFamily opened stored table /home/jangjk/SKIRT_DIR/SKIRT9/resources/SKIRT9_Resources_Core/SED/BruzualCharlotSEDFamily_Chabrier_hr.stab
21/07/2023 15:56:30.487   ParticleSource reads smoothed source particles from text file /home/jangjk/SKIRT_DIR/rur_dir/testing//part_old.txt...
21/07/2023

21/07/2023 16:04:08.789   Evaluation for level 8: 99.7%
21/07/2023 16:04:10.078   Subdividing level 9: 972376 nodes
21/07/2023 16:04:15.116   Evaluation for level 9: 13.4%
21/07/2023 16:04:20.426   Evaluation for level 9: 30.3%
21/07/2023 16:04:25.480   Evaluation for level 9: 43.7%
21/07/2023 16:04:30.586   Evaluation for level 9: 55.3%
21/07/2023 16:04:35.628   Evaluation for level 9: 68.7%
21/07/2023 16:04:40.980   Evaluation for level 9: 85.0%
21/07/2023 16:04:46.092   Evaluation for level 9: 98.4%
21/07/2023 16:04:51.746   Subdividing level 10: 1065928 nodes
21/07/2023 16:04:56.885   Evaluation for level 10: 9.4%
21/07/2023 16:05:01.885   Evaluation for level 10: 18.4%
21/07/2023 16:05:07.136   Evaluation for level 10: 30.9%
21/07/2023 16:05:12.194   Evaluation for level 10: 41.6%
21/07/2023 16:05:17.578   Evaluation for level 10: 47.2%
21/07/2023 16:05:22.596   Evaluation for level 10: 56.6%
21/07/2023 16:05:27.666   Evaluation for level 10: 66.6%
21/07/2023 16:05:32.804   Evalua

21/07/2023 16:09:02.822   Launched primary emission photon packets: 0.5%
21/07/2023 16:09:07.856   Launched primary emission photon packets: 1.2%
21/07/2023 16:09:13.016   Launched primary emission photon packets: 1.8%
21/07/2023 16:09:18.064   Launched primary emission photon packets: 2.4%
21/07/2023 16:09:23.068   Launched primary emission photon packets: 2.9%
21/07/2023 16:09:28.231   Launched primary emission photon packets: 3.5%
21/07/2023 16:09:33.243   Launched primary emission photon packets: 4.0%
21/07/2023 16:09:38.278   Launched primary emission photon packets: 4.5%
21/07/2023 16:09:43.280   Launched primary emission photon packets: 5.1%
21/07/2023 16:09:48.306   Launched primary emission photon packets: 5.6%
21/07/2023 16:09:53.360   Launched primary emission photon packets: 6.2%
21/07/2023 16:09:58.376   Launched primary emission photon packets: 6.7%
21/07/2023 16:10:03.384   Launched primary emission photon packets: 7.2%
21/07/2023 16:10:08.387   Launched primary emission

21/07/2023 16:17:26.983   SEDInstrument rot_000_000_10 wrote SED to /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_000_000_10_sed.dat
21/07/2023 16:17:27.016   SEDInstrument rot_000_000_15 wrote SED to /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_000_000_15_sed.dat
21/07/2023 16:17:27.049   SEDInstrument rot_000_000_20 wrote SED to /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_000_000_20_sed.dat
21/07/2023 16:17:27.175   FullInstrument rot_010_000 wrote SED to /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_010_000_sed.dat
21/07/2023 16:17:27.334   FullInstrument rot_010_000 wrote total flux to FITS file /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_010_000_total.fits
21/07/2023 16:17:27.411   FullInstrument rot_010_000 wrote transparent flux to FITS file /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_010_000_transparent.fits
21/07/2023 16:17:27.486   FullInstrument rot_010_000 wrote primarydirect flux to FITS file /home/ja

21/07/2023 16:17:30.844   SEDInstrument rot_060_000_10 wrote SED to /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_060_000_10_sed.dat
21/07/2023 16:17:30.877   SEDInstrument rot_060_000_15 wrote SED to /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_060_000_15_sed.dat
21/07/2023 16:17:30.910   SEDInstrument rot_060_000_20 wrote SED to /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_060_000_20_sed.dat
21/07/2023 16:17:31.023   FullInstrument rot_070_000 wrote SED to /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_070_000_sed.dat
21/07/2023 16:17:31.179   FullInstrument rot_070_000 wrote total flux to FITS file /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_070_000_total.fits
21/07/2023 16:17:31.258   FullInstrument rot_070_000 wrote transparent flux to FITS file /home/jangjk/SKIRT_DIR/rur_dir/testing/result/nh_ext_rot_070_000_transparent.fits
21/07/2023 16:17:31.331   FullInstrument rot_070_000 wrote primarydirect flux to FITS file /home/ja